# Constitutional Bible - QBIT Law Analysis

Interactive analysis of the mall's constitutional history.

**Purpose:** Query how laws evolved, how QBIT drifted, what the world was like at different moments.

**Data Source:** `data/constitution_log.jsonl` - written by `law_system.py`

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pathlib import Path

# Set plotting style
sns.set_theme(style="darkgrid")
plt.rcParams['figure.figsize'] = (12, 6)

## Load Constitutional Log

In [ ]:
def load_constitution_log(log_path: str = "../data/constitution_log.jsonl") -> pd.DataFrame:
    """Load JSONL log into pandas DataFrame."""
    entries = []
    
    with open(log_path, 'r') as f:
        for line in f:
            entry = json.loads(line)
            
            # Flatten for easier analysis
            flat_entry = {
                'timestamp': entry['timestamp'],
                'law_id': entry['law']['law_id'],
                'law_title': entry['law']['title'],
                'political_power': entry['political_power'],
                'strength': entry['strength'],
                'interpretation_radius': entry['interpretation_radius'],
                
                # QBIT weights (what the law wants)
                'weight_heat': entry['law']['qbit_weights'].get('heat', 0),
                'weight_debt': entry['law']['qbit_weights'].get('debt', 0),
                'weight_coherence': entry['law']['qbit_weights'].get('coherence', 0),
                'weight_gravity': entry['law']['qbit_weights'].get('gravity', 0),
                
                # QBIT snapshot (what the world was like)
                'qbit_global_heat': entry['qbit_snapshot'].get('global_heat', 0),
                'qbit_global_debt': entry['qbit_snapshot'].get('global_debt', 0),
                'qbit_global_coherence': entry['qbit_snapshot'].get('global_coherence', 0),
                
                # Store full objects for detailed queries
                '_full_law': entry['law'],
                '_full_qbit': entry['qbit_snapshot'],
                '_power_breakdown': entry['power_breakdown']
            }
            
            entries.append(flat_entry)
    
    df = pd.DataFrame(entries)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')
    
    return df

# Load the log
constitution = load_constitution_log()
print(f"Loaded {len(constitution)} constitutional entries")
constitution.head()

## Timeline: When Were Laws Enacted?

In [ ]:
# Chronological timeline
fig, ax = plt.subplots(figsize=(14, 6))

# Plot each law as a point on timeline
ax.scatter(
    constitution['timestamp'],
    constitution['political_power'],
    s=constitution['strength'] * 100,  # Size = strength
    c=constitution['interpretation_radius'],  # Color = wiggle room
    cmap='RdYlGn',
    alpha=0.6,
    edgecolors='black'
)

# Annotate law titles
for idx, row in constitution.iterrows():
    ax.annotate(
        row['law_title'],
        (row['timestamp'], row['political_power']),
        fontsize=8,
        rotation=15,
        ha='left'
    )

ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Time')
ax.set_ylabel('Political Power')
ax.set_title('Constitutional Timeline\n(size=strength, color=interpretation freedom)')
plt.colorbar(ax.collections[0], label='Interpretation Radius', ax=ax)
plt.tight_layout()
plt.show()

## QBIT Drift Over Time

How have global QBIT values changed as laws were enacted?

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

qbit_dimensions = [
    ('qbit_global_heat', 'Global Heat'),
    ('qbit_global_debt', 'Global Debt'),
    ('qbit_global_coherence', 'Global Coherence'),
]

for i, (col, label) in enumerate(qbit_dimensions):
    ax = axes[i]
    
    # Plot QBIT dimension over time
    ax.plot(
        constitution['timestamp'],
        constitution[col],
        marker='o',
        linewidth=2,
        markersize=8
    )
    
    # Mark law enactments
    for idx, row in constitution.iterrows():
        ax.axvline(row['timestamp'], color='red', alpha=0.2, linestyle='--')
    
    ax.set_xlabel('Time')
    ax.set_ylabel(label)
    ax.set_title(f'{label} Evolution')
    ax.grid(True, alpha=0.3)

# Use last subplot for summary
axes[3].axis('off')
summary_text = f"""
Constitutional Summary
━━━━━━━━━━━━━━━━━━━━━━
Laws Enacted: {len(constitution)}
Time Span: {constitution['timestamp'].min()} to {constitution['timestamp'].max()}

Current QBIT State:
  Heat: {constitution['qbit_global_heat'].iloc[-1]:.3f}
  Debt: {constitution['qbit_global_debt'].iloc[-1]:.3f}
  Coherence: {constitution['qbit_global_coherence'].iloc[-1]:.3f}

Strongest Law: {constitution.loc[constitution['strength'].idxmax(), 'law_title']}
Most Controversial: {constitution.loc[constitution['interpretation_radius'].idxmax(), 'law_title']}
"""
axes[3].text(0.1, 0.5, summary_text, fontsize=11, family='monospace', va='center')

plt.tight_layout()
plt.show()

## Law Relevance Analysis

Which laws are becoming obsolete as QBIT drifts?

In [ ]:
def calculate_law_relevance(constitution_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate how relevant each law is to current QBIT state.
    
    Relevance = how much has the world changed since law was enacted?
    """
    if len(constitution_df) == 0:
        return constitution_df
    
    # Get current QBIT state (last entry)
    current = constitution_df.iloc[-1]
    current_heat = current['qbit_global_heat']
    current_debt = current['qbit_global_debt']
    current_coherence = current['qbit_global_coherence']
    
    relevance_scores = []
    
    for idx, law in constitution_df.iterrows():
        # Calculate QBIT drift since enactment
        drift_heat = abs(current_heat - law['qbit_global_heat'])
        drift_debt = abs(current_debt - law['qbit_global_debt'])
        drift_coherence = abs(current_coherence - law['qbit_global_coherence'])
        
        total_drift = drift_heat + drift_debt + drift_coherence
        
        # Relevance: 1.0 = no drift, 0.0 = massive drift
        relevance = 1.0 / (1.0 + total_drift)
        
        relevance_scores.append({
            'law_id': law['law_id'],
            'law_title': law['law_title'],
            'total_drift': total_drift,
            'relevance': relevance,
            'status': (
                '✓ Still Relevant' if relevance > 0.7 else
                '⚠ Becoming Outdated' if relevance > 0.4 else
                '✗ Obsolete'
            )
        })
    
    return pd.DataFrame(relevance_scores)

relevance = calculate_law_relevance(constitution)
relevance = relevance.sort_values('relevance', ascending=False)

print("Law Relevance to Current QBIT State:")
print("="*60)
relevance[['law_title', 'relevance', 'total_drift', 'status']]

## Political Power Distribution

Which actors/factions supported which laws?

In [ ]:
# Extract power breakdown for all laws
def analyze_power_distribution(constitution_df: pd.DataFrame):
    """
    Show which actors supported/opposed which laws.
    """
    all_actors = set()
    
    # Collect all unique actors
    for breakdown in constitution_df['_power_breakdown']:
        all_actors.update(breakdown.keys())
    
    # Build matrix: laws × actors
    matrix_data = []
    
    for idx, row in constitution_df.iterrows():
        law_powers = {}
        for actor in all_actors:
            law_powers[actor] = row['_power_breakdown'].get(actor, 0)
        law_powers['law_title'] = row['law_title']
        matrix_data.append(law_powers)
    
    matrix_df = pd.DataFrame(matrix_data)
    matrix_df = matrix_df.set_index('law_title')
    
    # Plot heatmap
    plt.figure(figsize=(12, len(constitution_df) * 0.8))
    sns.heatmap(
        matrix_df,
        annot=True,
        fmt='.2f',
        cmap='RdYlGn',
        center=0,
        cbar_kws={'label': 'Political Power'},
        linewidths=0.5
    )
    plt.title('Political Power Distribution: Actors × Laws')
    plt.ylabel('Law')
    plt.xlabel('Actor/Faction')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return matrix_df

power_matrix = analyze_power_distribution(constitution)
power_matrix

## Query: "What was the world like when X law passed?"

In [ ]:
def describe_world_at_law_enactment(law_title: str, constitution_df: pd.DataFrame):
    """
    Narrative description of QBIT state when a law was enacted.
    
    This is what an AI constructor or LLM can query to understand context.
    """
    law = constitution_df[constitution_df['law_title'] == law_title].iloc[0]
    
    print(f"\n{'='*70}")
    print(f"LAW: {law['law_title']}")
    print(f"{'='*70}\n")
    
    print(f"📅 Enacted: {law['timestamp']}")
    print(f"⚖️  Political Power: {law['political_power']:.3f}")
    print(f"💪 Strength: {law['strength']:.3f}")
    print(f"🎯 Interpretation Radius: {law['interpretation_radius']:.3f}")
    print()
    
    print("🌍 QBIT STATE AT ENACTMENT:")
    print(f"   Heat (activity): {law['qbit_global_heat']:.3f}")
    print(f"   Debt (economic pressure): {law['qbit_global_debt']:.3f}")
    print(f"   Coherence (order): {law['qbit_global_coherence']:.3f}")
    print()
    
    print("📊 LAW'S QBIT DIRECTION:")
    print(f"   Heat weight: {law['weight_heat']:+.2f}")
    print(f"   Debt weight: {law['weight_debt']:+.2f}")
    print(f"   Coherence weight: {law['weight_coherence']:+.2f}")
    print(f"   Gravity weight: {law['weight_gravity']:+.2f}")
    print()
    
    print("🗳️  POLITICAL SUPPORT:")
    for actor, power in law['_power_breakdown'].items():
        emoji = '✓' if power > 0 else '✗'
        print(f"   {emoji} {actor}: {power:+.3f}")
    print()
    
    # Generate narrative
    heat_desc = "high activity" if law['qbit_global_heat'] > 0.6 else "low activity" if law['qbit_global_heat'] < 0.4 else "moderate activity"
    debt_desc = "economic stress" if law['qbit_global_debt'] > 0.6 else "economic stability" if law['qbit_global_debt'] < 0.4 else "balanced economy"
    coherence_desc = "ordered" if law['qbit_global_coherence'] > 0.6 else "chaotic" if law['qbit_global_coherence'] < 0.4 else "mixed order"
    
    print("📖 NARRATIVE:")
    print(f"   When '{law['law_title']}' was enacted, the mall was experiencing")
    print(f"   {heat_desc}, {debt_desc}, and {coherence_desc} conditions.")
    
    if law['political_power'] > 0:
        print(f"   The law had strong political support ({law['political_power']:.2f}) and")
    else:
        print(f"   The law faced political opposition ({law['political_power']:.2f}) but")
    
    if law['interpretation_radius'] < 0.3:
        print(f"   was enforced strictly with little room for interpretation.")
    elif law['interpretation_radius'] < 0.6:
        print(f"   allowed moderate creative interpretation.")
    else:
        print(f"   was loosely enforced with significant wiggle room.")
    print()

# Example query
if len(constitution) > 0:
    describe_world_at_law_enactment(constitution.iloc[0]['law_title'], constitution)

## Predictive: Which Laws Might Pass Next?

Based on current QBIT state, score hypothetical laws.

In [ ]:
def score_hypothetical_law(law_weights: dict, current_qbit: dict) -> float:
    """
    Score a hypothetical law against current QBIT state.
    
    Returns: predicted political power if this law were proposed today.
    """
    score = 0.0
    
    for dim, weight in law_weights.items():
        qbit_key = f'qbit_global_{dim}'
        if qbit_key in current_qbit:
            score += weight * current_qbit[qbit_key]
    
    return score

# Get current QBIT state
if len(constitution) > 0:
    current_state = constitution.iloc[-1]
    
    # Define hypothetical laws
    hypothetical_laws = [
        {
            'title': 'Extended Mall Hours',
            'weights': {'heat': 0.5, 'debt': 0.3, 'coherence': -0.2}
        },
        {
            'title': 'Increased Security Patrols',
            'weights': {'heat': -0.3, 'debt': 0.1, 'coherence': 0.6}
        },
        {
            'title': 'Rent Reduction for Vendors',
            'weights': {'heat': 0.2, 'debt': -0.5, 'coherence': -0.1}
        },
    ]
    
    print("\nHYPOTHETICAL LAW PREDICTIONS (based on current QBIT):")
    print("="*60)
    
    predictions = []
    for hyp_law in hypothetical_laws:
        score = score_hypothetical_law(hyp_law['weights'], current_state)
        likelihood = "High" if score > 0.3 else "Medium" if score > 0 else "Low"
        
        predictions.append({
            'Law': hyp_law['title'],
            'Predicted Power': f"{score:.3f}",
            'Likelihood': likelihood
        })
        
    pd.DataFrame(predictions)

## Export for AI Constructors

Generate JSON summary that AI systems can query.

In [ ]:
def export_constitutional_summary(constitution_df: pd.DataFrame, output_path: str):
    """
    Export AI-readable summary of constitutional state.
    
    This is what LLM constructors / behavior trees query.
    """
    if len(constitution_df) == 0:
        return
    
    current = constitution_df.iloc[-1]
    
    summary = {
        'current_timestamp': current['timestamp'].isoformat(),
        
        'current_qbit': {
            'heat': float(current['qbit_global_heat']),
            'debt': float(current['qbit_global_debt']),
            'coherence': float(current['qbit_global_coherence'])
        },
        
        'active_laws': [
            {
                'law_id': row['law_id'],
                'title': row['law_title'],
                'strength': float(row['strength']),
                'interpretation_radius': float(row['interpretation_radius']),
                'enacted': row['timestamp'].isoformat(),
                'effects': row['_full_law']['effects']
            }
            for idx, row in constitution_df.iterrows()
        ],
        
        'relevance_analysis': relevance.to_dict('records') if 'relevance' in dir() else [],
        
        'meta': {
            'total_laws': len(constitution_df),
            'oldest_law': constitution_df.iloc[0]['law_title'],
            'newest_law': constitution_df.iloc[-1]['law_title'],
            'average_strength': float(constitution_df['strength'].mean()),
            'average_interpretation': float(constitution_df['interpretation_radius'].mean())
        }
    }
    
    with open(output_path, 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"✓ Exported constitutional summary to {output_path}")
    return summary

if len(constitution) > 0:
    summary = export_constitutional_summary(constitution, '../data/constitution_summary.json')
    print("\nAI constructors can now query:")
    print("  - current_qbit: What's the world like now?")
    print("  - active_laws: What rules are in force?")
    print("  - interpretation_radius: How creative can I be?")
    print("  - relevance_analysis: Which laws are obsolete?")

## Custom Queries

Add your own analysis here. The constitutional log is your game bible - query anything!

In [ ]:
# Example: Find laws enacted during high chaos (low coherence)
chaotic_laws = constitution[constitution['qbit_global_coherence'] < 0.4]
print(f"Laws enacted during chaotic periods: {len(chaotic_laws)}")
chaotic_laws[['law_title', 'timestamp', 'qbit_global_coherence']]

In [ ]:
# Example: Which law had most interpretation freedom?
most_flexible = constitution.loc[constitution['interpretation_radius'].idxmax()]
print(f"Most flexible law: {most_flexible['law_title']}")
print(f"Interpretation radius: {most_flexible['interpretation_radius']:.3f}")
print(f"\nThis law allows maximum creative interpretation by AI constructors.")